In [ ]:
import numpy as np
from torch_obb.estimation import obb_estimate_pca

rng = np.random.RandomState(42)
points = rng.random((200, 3)).astype(np.float32) * np.array([5, 2, 1])
offsets = np.array([0, points.shape[0]])


vertices = obb_estimate_pca(points, offsets).squeeze()



In [94]:
points_np = points_wp.numpy()

centroids_np = points_np.mean(axis=0)
print("centroids_np", centroids_np)

centered_np = points_np - centroids_np
cov_np = np.matmul(centered_np.T, centered_np) / points_np.shape[0]

eigvals, eigvecs = np.linalg.eigh(cov_np)

if np.linalg.det(eigvecs) < 0:
    print("inverting eigvecs")
    eigvecs[:, 2] *= -1.0

print("eigenvecs", eigvecs)
print("eigenvals", eigvals)

rotated = centered_np @ eigvecs
min_extent = rotated.min(axis=0)
max_extent = rotated.max(axis=0)
extent = max_extent - min_extent

#q = (min_extent + max_extent) * 0.5
#center = eigvecs @ q

vertices_np = np.array(AABB_VERTICES)
vertices_np = vertices_np - 0.5
vertices_np = vertices_np * extent

print("vertices_np", vertices_np)

vertices_np = centroids_np + vertices_np @ eigvecs.T

print("min_extent", min_extent)
print("max_extent", max_extent)
print("vertices_np", vertices_np)


centroids_np [2.5286605 1.0418893 0.4823775]
eigenvecs [[-0.00699913 -0.03880151 -0.9992224 ]
 [-0.03494049  0.9986462  -0.03853439]
 [ 0.9993649   0.03464362 -0.0083454 ]]
eigenvals [0.09101889 0.35623613 2.1207962 ]
vertices_np [[-0.51245195 -1.02988863 -2.49141645]
 [-0.51245195 -1.02988863  2.49141645]
 [-0.51245195  1.02988863 -2.49141645]
 [-0.51245195  1.02988863  2.49141645]
 [ 0.51245195 -1.02988863 -2.49141645]
 [ 0.51245195 -1.02988863  2.49141645]
 [ 0.51245195  1.02988863 -2.49141645]
 [ 0.51245195  1.02988863  2.49141645]]
min_extent [-0.50979656 -1.0721182  -2.5052774 ]
max_extent [0.51510733 0.98765916 2.4775555 ]
vertices_np [[ 5.06168761e+00  1.27305475e-01 -4.46361932e-02]
 [ 8.27293614e-02 -6.47049414e-02 -8.62199315e-02]
 [ 4.98176515e+00  2.18429421e+00  2.67219403e-02]
 [ 2.80690132e-03  1.99228379e+00 -1.48617980e-02]
 [ 5.05451417e+00  9.14948284e-02  9.79616797e-01]
 [ 7.55559235e-02 -1.00515588e-01  9.38033058e-01]
 [ 4.97459171e+00  2.14848356e+00  1.0509749

In [95]:
import torch

points_t = torch.from_numpy(points_np)
points_t_jagged = torch.nested.as_nested_tensor([points_t], dtype=torch.float32, device=torch.device("cpu"), layout=torch.jagged)

centroids_t = torch.mean(points_t_jagged, dim=1)
centered_t_jagged = points_t_jagged - centroids_t

npoints_t = points_t_jagged.offsets().diff()

cov_t = torch.bmm(centered_t_jagged.mT, centered_t_jagged) / npoints_t

eigvals_t, R_t = torch.linalg.eigh(cov_t)
rotated_t_jagged = torch.bmm(centered_t_jagged, R_t)
min_extent_t = rotated_t_jagged.min(dim=1).values
max_extent_t = rotated_t_jagged.max(dim=1).values
extent_t = max_extent_t - min_extent_t

vertices_t = torch.tensor(AABB_VERTICES).unsqueeze(0)
vertices_t = vertices_t - 0.5
vertices_t = vertices_t * extent_t
vertices_t = centroids_t + torch.bmm(vertices_t, R_t.mT)
vertices_np = vertices_t.numpy().squeeze()


In [101]:
vertices_t.is_nested

False

In [97]:
vertices_np

array([[ 5.06168747e+00,  1.27305925e-01, -4.46361899e-02],
       [ 8.27283859e-02, -6.47046566e-02, -8.62199664e-02],
       [ 4.98176479e+00,  2.18429446e+00,  2.67219543e-02],
       [ 2.80594826e-03,  1.99228382e+00, -1.48617625e-02],
       [ 5.05451393e+00,  9.14952159e-02,  9.79616582e-01],
       [ 7.55550861e-02, -1.00515366e-01,  9.38032866e-01],
       [ 4.97459173e+00,  2.14848375e+00,  1.05097485e+00],
       [-4.36735153e-03,  1.95647311e+00,  1.00939107e+00]], dtype=float32)

In [7]:
import pygfx as gfx
import trimesh
from rendercanvas.glfw import GlfwRenderCanvas
_TRIS = [[1, 3, 0],
         [4, 1, 0],
         [0, 3, 2],
         [2, 4, 0],
         [1, 7, 3],
         [5, 1, 4],
         [5, 7, 1],
         [3, 7, 2],
         [6, 4, 2],
         [2, 7, 6],
         [6, 5, 4],
         [7, 5, 6]]
box = trimesh.Trimesh(
    vertices=vertices, faces=np.array(_TRIS), process=False)

canvas = GlfwRenderCanvas(size=(1024, 768), max_fps=60, update_mode="continuous")   


scene = gfx.Scene()
background = gfx.Background(None, gfx.BackgroundMaterial('white'))
scene.add(background)

points_geometry = gfx.Geometry(positions=points.astype(np.float32))
pc = gfx.Points(points_geometry, material=gfx.PointsMaterial(size=10, color=(0, 0, 0, 1.0)))
scene.add(pc)
box_local = gfx.BoxHelper(thickness=2, color="green")
box_local.set_transform_by_object(pc, space="local")
pc.add(box_local)  # note that the parent is `mesh` here, not `scene`


obb_geometry = gfx.Geometry(positions=vertices.squeeze().astype(np.float32), indices=np.array(_TRIS, dtype=np.int32))

obb = gfx.Mesh(obb_geometry, gfx.MeshBasicMaterial(wireframe=True, color='red'))
scene.add(obb)


# obb_geometry_wp = gfx.Geometry(positions=vertices_out.astype(np.float32), indices=np.array(_TRIS, dtype=np.int32))

# obb_wp = gfx.Mesh(obb_geometry_wp, gfx.MeshBasicMaterial(wireframe=True, color='cyan'))
# scene.add(obb_wp)

disp = gfx.Display(canvas=canvas)
disp.show(scene)


canvas.is_closed() is deprecated, use canvas.get_closed() instead.
Your scene does not contain any lights. Some objects may not be visible.
Unable to find extension: VK_EXT_physical_device_drm
Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.
Unrecognized present mode 1000361000
Unrecognized present mode 1000361000


Unrecognized present mode 1000361000
